In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import os

In [ ]:
data_src = r'../dat/bundesnetzagentur/pre_processed_data/prod_price_de_lu_2019_2021.csv'

In [ ]:
df = pd.read_csv(data_src,
                 parse_dates=['time'])
df = df.set_index('time')

renewable_cols = ['Biomass', 
                  'Hydropower', 
                  'Offshore Wind', 
                  'Onshore Wind', 
                  'Solar', 
                  'Other Renewables']

non_renewable_cols = ['Nuclear', 
                      'Brown Coal', 
                      'Black Coal',
                      'Natural Gas',
                      'Other Conventional Sources']

df = df.resample('D').mean()
df['Total renewable'] = df[renewable_cols].sum(axis=1)
df['Total non-renewable'] = df[non_renewable_cols].sum(axis=1)
df['Total'] = df[['Total renewable', 'Total non-renewable']].sum(axis=1)

In [ ]:
X_renewable = df['Total renewable'].to_numpy().reshape(-1,1)
X_non_renewable = df['Total non-renewable'].to_numpy().reshape(-1,1)
y = df['Price'].to_numpy()

reg_renewable = LinearRegression()
reg_non_renewable = LinearRegression()

reg_renewable.fit(X_renewable, y)
reg_non_renewable.fit(X_non_renewable, y)

m_renewable = reg_renewable.coef_
b_renewable = reg_renewable.intercept_

m_non_renewable = reg_non_renewable.coef_
b_non_renewable = reg_non_renewable.intercept_

plt.figure(figsize=(20,10))

plt.plot(X_renewable, y, 'o', label='Renewable')
plt.plot(X_renewable, m_renewable*X_renewable + b_renewable, 'g', linewidth=3, label='Regression Line (Renewable)')

plt.plot(X_non_renewable, y, 'o', label='Conventional')
plt.plot(X_non_renewable, m_non_renewable*X_non_renewable + b_non_renewable, 'r', linewidth=3, label='Regression Line (Conventional)')

plt.xlabel('Energy Production [MWh]')
plt.ylabel('Price [€/MWh]')

plt.tight_layout()
plt.legend()
# plt.savefig('regression_test.pdf')
plt.show()

In [ ]:
df = df.resample('M').mean()
df = df[['Total renewable', 'Total non-renewable', 'Total', 'Price']]
df['Percentage renewable'] = df['Total renewable'] / df['Total']
df['Percentage non-renewable'] = df['Total non-renewable'] / df['Total']
df.head()

In [ ]:
fig, ax1 = plt.subplots(figsize=(15,5))

ax2 = ax1.twinx()

ax1.plot(df['Price'], color='blue')
ax1.set_ylabel('Preis [€/MWh]')

ax2.plot(df['Percentage renewable'], color='orange')
ax2.set_ylabel('Percentage Renewable Production [%]')

# Plots the non-renawable energy
#plt.plot(df['Percentage non-renewable'], color='red')

plt.plot()

In [ ]:
# smooth the curves
start_date = '2019-01-31'
end_date = '2021-12-31'

index_daily = pd.date_range(start_date, end_date, freq='1H')
df_smooth = df.reindex(index=index_daily).interpolate('cubic')

fig, ax1 = plt.subplots(figsize=(15,5))

ax2 = ax1.twinx()

ax1.plot(df_smooth['Price'], color='blue', label='Price')
ax1.set_ylabel('Preis [€/MWh]')

ax2.plot(df_smooth['Percentage renewable'], color='orange', label='Percentage Renewable')
ax2.set_ylabel('Percentage Renewable Production [%]')

# Plots the non-renawable energy
#plt.plot(df_smooth['Percentage non-renewable'], color='red')

plt.plot()

# The plot contains the percentage of the renewable energy of the total produced energy and the price.
# It is clearly visible that if the percentage of renewable increases the price decreases and vice versa. 

In [ ]:
percentage_df = df[['Percentage renewable',
                    'Percentage non-renewable',
                    'Price']]

X = percentage_df.index
percentage_renewable = percentage_df['Percentage renewable'].to_numpy()
percentage_non_renewable = percentage_df['Percentage non-renewable'].to_numpy() 

width = 15

fig, ax = plt.subplots(figsize=(15, 10))
ax2 = ax.twinx()


ax.set_xlabel('Date')

ax.bar(X, percentage_renewable, width=width)
ax.bar(X, percentage_non_renewable, bottom=percentage_renewable, width=width)
ax2.plot(percentage_df['Price'],
         color='k',
        linewidth=3, 
        marker='o')

ax.legend(['Percentage of renewable', 'Percentage of non-renewable'], loc='upper left')
ax2.legend(['Price'], loc='upper center')

ax.set_ylabel('Percentage')
ax2.set_ylabel('Price')




plt.tight_layout()
plt.show()


In [ ]:
# example day
df = pd.read_csv(data_src,
                 parse_dates=['time'])
df = df.set_index('time')

renewable_cols = ['Biomass', 
                  'Hydropower', 
                  'Offshore Wind', 
                  'Onshore Wind', 
                  'Solar', 
                  'Other Renewables']

non_renewable_cols = ['Nuclear', 
                      'Brown Coal', 
                      'Black Coal',
                      'Natural Gas',
                      'Other Conventional Sources']

df['Total renewable'] = df[renewable_cols].sum(axis=1)
df['Total non-renewable'] = df[non_renewable_cols].sum(axis=1)
df['Total'] = df[['Total renewable', 'Total non-renewable']].sum(axis=1)

df = df[['Total renewable', 'Total non-renewable', 'Total', 'Price']]
df['Percentage renewable'] = df['Total renewable'] / df['Total']
df['Percentage non-renewable'] = df['Total non-renewable'] / df['Total']

percentage_df = df[['Percentage renewable',
                    'Percentage non-renewable',
                    'Price']]

percentage_df = percentage_df.loc['01-03-2021']

X = percentage_df.index.hour

percentage_renewable = percentage_df['Percentage renewable'].to_numpy()
percentage_non_renewable = percentage_df['Percentage non-renewable'].to_numpy() 

width = 4

fig, ax = plt.subplots(figsize=(10, 5))
ax2 = ax.twinx()

ax.set_xlabel('Hour of the day')

ax.plot(percentage_df['Percentage renewable'],
       color='blue',
       linewidth=3,
       marker='x')

ax2.plot(percentage_df['Price'],
         color='k',
        linewidth=3, 
        marker='x')

fig.legend(['Percentage renewable', 'Price'], loc='upper left', bbox_to_anchor=(0.08,0.97))

ax.set_ylabel('Percentage of renewable')
ax2.set_ylabel('Price [€/MWh]')

x_labels = list(range(24))

plt.xticks(percentage_df.index)
ax.set_xticklabels(x_labels)

plt.tight_layout()
plt.savefig(os.path.join(r'../doc/fig/', 'example_day.pdf'))

plt.show()
